## 存取當下日期及計算三年前日期

In [3]:
import datetime
from dateutil.relativedelta import relativedelta

# 當下的年、月、日
now_year, now_month, now_day = str(datetime.datetime.now()).split(' ')[0].split('-')
timestamp_now = int(datetime.datetime(int(now_year), int(now_month), int(now_day), 0, 0).timestamp())

# 令開始時間為3年前
start_timestamp = int((datetime.datetime(int(now_year), int(now_month), int(now_day), 0, 0) - relativedelta(years=3)).timestamp())

#datetime.datetime(2016, 2, 29, 0, 0) - relativedelta(years=3)

## 製造該ETF當下的網址  (來源: yahoo finance)

In [4]:
def url_of_ETF_now_wk(etf_name):  # 週資料
    url='https://finance.yahoo.com/quote/ETF_NAME/history?period1=START_TIME&period2=NOW_TIME&interval=1wk&filter=history&frequency=1wk.html'
   
    exact_url=url.replace('ETF_NAME', str(etf_name))
    exact_url=exact_url.replace('START_TIME', str(start_timestamp))
    exact_url=exact_url.replace('NOW_TIME', str(timestamp_now))
    
    return exact_url

def url_of_ETF_now_mo(etf_name):  # 月資料
    url='https://finance.yahoo.com/quote/ETF_NAME/history?period1=START_TIME&period2=NOW_TIME&interval=1mo&filter=history&frequency=1mo.html'
   
    exact_url=url.replace('ETF_NAME', str(etf_name))
    exact_url=exact_url.replace('START_TIME', str(start_timestamp))
    exact_url=exact_url.replace('NOW_TIME', str(timestamp_now))
    
    return exact_url

#url_of_ETF_now_mo('LQD')

## 讀標的ETF的名子

In [5]:
import csv

goal_ETF = []
with open('goal_ETF.csv', newline='') as f:
    reader = csv.reader(f)
    for row in reader:
        goal_ETF.append(row[0])
print(goal_ETF)

['LQD', 'VCSH', 'VCIT', 'IGSB', 'IGIB', 'SPSB', 'SPIB', 'USIG', 'VCLT', 'GSY', 'SLQD', 'ICSH', 'CORP', 'IGLB', 'SPLB', 'IGHG', 'LQDH', 'QLTA', 'PFIG', 'SKOR', 'IGBH', 'CBND', 'LKOR', 'LDRI', 'IGIH']


## 載入網頁內容

In [6]:
import requests
# 使用 requests.get 把網頁內容載下來

def get_response_wk(etf_name):
    response = requests.get(url_of_ETF_now_wk(etf_name))
    response.text  # 展示載來的資料
    # 使用 split 這個函數 把文字做切割成不同行
    lines = response.text.split('{"date":')
    return lines

def get_response_mo(etf_name):
    response = requests.get(url_of_ETF_now_mo(etf_name))
    response.text

    lines = response.text.split('{"date":')
    return lines

## 定義爬ETF價格函數

In [7]:
def get_price_w(etf_name):    #取得該ETF各週價格
    
    newlines = []
    price = []        # 收集價格
    
    for line in get_response_wk(etf_name):
        line = line.split('}')[0]             # 取'}'前的部分
        if len(line.split(',')) == 7:        # 用「,」切欄位 看是否為7欄

            newlines.append(line)
            
    for i in range(len(newlines)-1):
        if newlines[i].split(',')[0].isdigit():     # 檢查非空白 
            try:
                adj_price = round(float(newlines[i].split(',')[6].split(':')[1][:-1]),3)
            except:
                adj_price = 0               
            if adj_price > 0:
                price.append(adj_price)

    return price

def get_price_m(etf_name):    #取得該ETF各月價格
    
    newlines = []
    price = []

            
    for line in get_response_mo(etf_name):
        line = line.split('}')[0]
        if len(line.split(',')) == 7:  

            newlines.append(line) 

    for i in range(len(newlines)-1):
        if newlines[i].split(',')[0].isdigit():
            try:
                adj_price = round(float(newlines[i].split(',')[6].split(':')[1][:-1]),4)
            except:
                adj_price = 0               
            if adj_price > 0:
                price.append(adj_price)
            
    return price

#print(get_price_m('LQD'))

## 存下各ETF的週與月價格

In [8]:
ETF_PRICE_wk = {}  # 月價格
ETF_PRICE_mo = {}  # 日價格


for ETF in goal_ETF:
    P = get_price_w(ETF)
    L = len(P)
    while L == 0:            # 偶而可能是連線問題出錯 要重新取得
        P = get_price_w(ETF)
        L = len(P)

    ETF_PRICE_wk.update({ETF: P})
    
for ETF in goal_ETF:
    P = get_price_m(ETF)
    L = len(P)
    while L == 0: 
        P = get_price_m(ETF)
        L = len(P)

    ETF_PRICE_mo.update({ETF: P})

## 計算各ETF週與月報酬

In [9]:
import numpy as np

Return_wk = {}
Return_mo = {}

for ETF in goal_ETF:
    wk_return = (np.array(ETF_PRICE_wk[ETF][:-1])-np.array(ETF_PRICE_wk[ETF][1:]))/np.array(ETF_PRICE_wk[ETF][1:])
    mo_return = (np.array(ETF_PRICE_mo[ETF][:-1])-np.array(ETF_PRICE_mo[ETF][1:]))/np.array(ETF_PRICE_mo[ETF][1:])
    
    Return_wk.update({ETF: wk_return})
    Return_mo.update({ETF: mo_return})

#for ETF in goal_ETF:
#   print(np.mean(Return_wk[ETF]))

## Generalized sharpe ratio

In [10]:
from math import sqrt
from scipy.stats import kurtosis, skew

Sharpe_ratio_wk = {}
Sharpe_ratio_mo = {}

rf = 2.38/100  # 美國公債殖利率
rf_wk = rf/52  #週化
rf_mo = rf/12  #月化

# 週資料部分 
for ETF in goal_ETF:
    mu = np.mean(Return_wk[ETF])
    sigma = sqrt(np.var(Return_wk[ETF]))
    Skew = skew(Return_wk[ETF])
    Kurt = kurtosis(Return_wk[ETF])
    
    try:
        t1 = 3*Kurt-4*Skew**2-9
        t2 = 3*Kurt-5*Skew**2-9
        alpha = 3*sqrt(t1)/(sigma**2*t2)
        beta = 3*Skew/(sigma*t2)
        eta = mu-3*Skew*sigma/t1
        delta = 3*sigma*sqrt(t2)/t1
        labd_alstr = beta+alpha(eta-rf_wk)/sqrt(delta**2+(eta-rf_wk)**2)
        phi = sqrt(alpha**2-beta**2)

        asksr = sqrt(2(labd_alstr(eta-rf_wk)-delta(phi-sqrt(alpha**2-(beta-labd_alstr)**2))))
        Sharpe_ratio_wk.update({ETF: asksr})
    
    except:
        delta_t = 1/52  # 1年52周
        b_3 = 2  # for logarithmic utility rho = 1, b_3 = (rho+1)/rho = 2
        SR = (mu-rf_wk)/sigma*sqrt(delta_t)
        
        assr = SR*sqrt(1+b_3*Skew/3*SR)
        Sharpe_ratio_wk.update({ETF: assr})
# 月資料部分       
for ETF in goal_ETF:
    mu = np.mean(Return_mo[ETF])
    sigma = sqrt(np.var(Return_mo[ETF]))
    Skew = skew(Return_mo[ETF])
    Kurt = kurtosis(Return_mo[ETF])
    
    try:
        t1 = 3*Kurt-4*Skew**2-9
        t2 = 3*Kurt-5*Skew**2-9
        alpha = 3*sqrt(t1)/(sigma**2*t2)
        beta = 3*Skew/(sigma*t2)
        eta = mu-3*Skew*sigma/t1
        delta = 3*sigma*sqrt(t2)/t1
        labd_alstr = beta+alpha(eta-rf_mo)/sqrt(delta**2+(eta-rf_mo)**2)
        phi = sqrt(alpha**2-beta**2)

        asksr = sqrt(2(labd_alstr(eta-rf_mo)-delta(phi-sqrt(alpha**2-(beta-labd_alstr)**2))))
        Sharpe_ratio_mo.update({ETF: asksr})
    
    except:
        delta_t = 1/12  # 1年12月
        b_3 = 2  # for logarithmic utility rho = 1, b_3 = (rho+1)/rho = 2
        SR = (mu-rf_mo)/sigma*sqrt(delta_t)
        
        assr = SR*sqrt(1+b_3*Skew/3*SR)
        Sharpe_ratio_mo.update({ETF: assr})

# Sharpe_ratio_wk,Sharpe_ratio_mo

## Sharpe Omega performance

In [11]:
L = 2.38/100  # 與前面 rf 一樣
L_wk = L/52
L_mo = L/12


def sharpe_omega(Return_dict, period):
    Omega = {}
    for ETF in list(Return_dict.keys()):
        ret_mean = np.mean(Return_dict[ETF])    # E(X)
        
        if period == 'wk':
            diff = L_wk-np.array(Return_dict[ETF])    # L-X
            n = 0          # amount of L-X > 0
            diff_mean = 0  # E(L-X)+
            for r in diff:
                if r > 0:              
                    diff_mean = (diff_mean*n + r)/(n + 1)
                    n = n + 1
            Omega.update({ETF:(ret_mean - L_wk)/diff_mean})
            
        elif period =='mo':
            diff = L_mo-np.array(Return_dict[ETF])
            n = 0
            diff_mean = 0
            for r in diff:
                if r > 0:              
                    diff_mean = (diff_mean*n + r)/(n + 1)
                    n = n + 1
            Omega.update({ETF:(ret_mean - L_mo)/diff_mean})
        else:
            print('Wrong period, only \'wk\' and \'mo\'')
            
    return Omega

# sharpe_omega(Return_wk, 'wk'),sharpe_omega(Return_mo, 'mo')

## Index of riskiness

In [72]:
from scipy.optimize import *

def riskiness_wk(ETF):
    def boundFunction1(a_g):
        F = sum(np.exp(-a_g*np.array(Return_wk[ETF])))-len(Return_wk[ETF])
        return F
    
    if np.mean(Return_wk[ETF]) > 0:
        guess = np.array([1])
    else:
        guess = np.array([-1])
        
    a_g = fsolve(boundFunction1, guess)
    
    return np.exp(-a_g[0])

def riskiness_mo(ETF):
    def boundFunction2(a_g):
        F = sum(np.exp(-a_g*np.array(Return_mo[ETF])))-len(Return_mo[ETF])
        return F
    
    if np.mean(Return_mo[ETF]) > 0:
        guess = np.array([1])
    else:
        guess = np.array([-1])
        
    a_g = fsolve(boundFunction2, guess)
    
    return np.exp(-a_g[0])

Riskiness_wk = {}
Riskiness_mo = {}

for ETF in goal_ETF:
    Riskiness_wk.update({ETF: riskiness_wk(ETF)})
    Riskiness_mo.update({ETF: riskiness_mo(ETF)})

# Riskness_wk, Riskness_mo

C:\Users\lawliet\Anaconda3\lib\site-packages\scipy\optimize\minpack.py:163: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  warnings.warn(msg, RuntimeWarning)


## 排序各個ETF指標值

In [73]:
import collections

# 排序
sorted_sha_wk = sorted(Sharpe_ratio_wk.items(), key=lambda x: x[1], reverse=True)
sorted_ome_wk = sorted(sharpe_omega(Return_wk, 'wk').items(), key=lambda x: x[1], reverse=True)
sorted_ris_wk = sorted(Riskiness_wk.items(), key=lambda x: x[1], reverse=True)

# 存key取得指標名稱
rank_sha_wk = collections.OrderedDict(sorted_sha_wk).keys()
rank_ome_wk = collections.OrderedDict(sorted_ome_wk).keys()
rank_ris_wk = collections.OrderedDict(sorted_ris_wk).keys()

# 月部分
sorted_sha_mo = sorted(Sharpe_ratio_mo.items(), key=lambda x: x[1], reverse=True)
sorted_ome_mo = sorted(sharpe_omega(Return_mo, 'mo').items(), key=lambda x: x[1], reverse=True)
sorted_ris_mo = sorted(Riskiness_mo.items(), key=lambda x: x[1], reverse=True)

rank_sha_mo = collections.OrderedDict(sorted_sha_mo).keys()
rank_ome_mo = collections.OrderedDict(sorted_ome_mo).keys()
rank_ris_mo = collections.OrderedDict(sorted_ris_mo).keys()

#rank_sha_wk,rank_ome_wk,rank_ris_wk
#rank_sha_mo,rank_ome_mo,rank_ris_mo

# 用Series存各排序
sh_w = pd.Series(np.array(list(rank_sha_wk)))
sh_m = pd.Series(np.array(list(rank_sha_mo)))
om_w = pd.Series(np.array(list(rank_ome_wk)))
om_m = pd.Series(np.array(list(rank_ome_mo)))
ri_w = pd.Series(np.array(list(rank_ris_wk)))
ri_m = pd.Series(np.array(list(rank_ris_mo)))

## 週資料分析結果

In [74]:
import pandas as pd

Rank_wk = {'Sharpe ratio':list(rank_sha_wk), 'Sharpe Omega':list(rank_ome_wk), 'Index of riskness':list(rank_ris_wk)}

dfwk = pd.DataFrame(Rank_wk)
dfwk.set_index(dfwk.index.values+1, inplace = True)
dfwk = dfwk.rename_axis('Ranking')
dfwk

,Sharpe ratio,Sharpe Omega,Index of riskness
Ranking,,,
1,LQDH,LQDH,IGBH
2,IGHG,IGHG,LQD
3,IGIB,IGIB,SPSB
4,VCLT,VCLT,LKOR
5,SPLB,SPLB,IGIH
6,USIG,USIG,VCLT
7,VCIT,VCIT,IGLB
8,CBND,CBND,IGIB
9,CORP,CORP,IGHG


## 週資料各指標排序相關係數

In [75]:
Cor_wk = {'Sha vs Omg':sh_w.corr(om_w, method='spearman'),
          'Omg vs Risk':om_w.corr(ri_w, method='spearman'),
          'Risk vs Sha':ri_w.corr(sh_w, method='spearman')}

pd.DataFrame(Cor_wk,index=['Spearman Corr.'])

C:\Users\lawliet\Anaconda3\lib\site-packages\scipy\stats\stats.py:245: RuntimeWarning: The input array could not be properly checked for nan values. nan values will be ignored.
  "values. nan values will be ignored.", RuntimeWarning)


,Sha vs Omg,Omg vs Risk,Risk vs Sha
Spearman Corr.,0.826923,-0.026154,0.008462


## 月資料分析結果

In [76]:
Rank_mo = {'Sharpe ratio':list(rank_sha_mo), 'Sharpe Omega':list(rank_ome_mo), 'Index of riskness':list(rank_ris_mo)}

df = pd.DataFrame(Rank_mo)
df.set_index(df.index.values+1, inplace = True)
df = df.rename_axis('Ranking')
df

,Sharpe ratio,Sharpe Omega,Index of riskness
Ranking,,,
1,IGLB,IGLB,VCLT
2,IGHG,LQDH,LDRI
3,LQDH,IGHG,VCIT
4,IGIH,SPSB,ICSH
5,SPSB,IGIH,LQD
6,IGIB,IGIB,SKOR
7,VCLT,VCLT,PFIG
8,SPLB,SPLB,VCSH
9,USIG,CBND,LKOR


## 月資料各指標排序相關係數

In [77]:
Cor_mo = {'Sha vs Omg':sh_m.corr(om_m, method='spearman'),
          'Omg vs Risk':om_m.corr(ri_m, method='spearman'),
          'Risk vs Sha':ri_m.corr(sh_m, method='spearman')}

pd.DataFrame(Cor_mo,index=['Spearman Corr.'])

C:\Users\lawliet\Anaconda3\lib\site-packages\scipy\stats\stats.py:245: RuntimeWarning: The input array could not be properly checked for nan values. nan values will be ignored.
  "values. nan values will be ignored.", RuntimeWarning)


,Sha vs Omg,Omg vs Risk,Risk vs Sha
Spearman Corr.,0.32,0.022308,0.087692


## 週資料與月資料比較之相關係數

In [79]:
Cor_wk_mo = {'Sharpe':sh_m.corr(sh_w, method='spearman'),
             'Omega':om_m.corr(om_w, method='spearman'),
             'Riskiness':ri_m.corr(ri_w, method='spearman')}
pd.DataFrame(Cor_wk_mo,index=['month vs week Spearman Corr.'])

C:\Users\lawliet\Anaconda3\lib\site-packages\scipy\stats\stats.py:245: RuntimeWarning: The input array could not be properly checked for nan values. nan values will be ignored.
  "values. nan values will be ignored.", RuntimeWarning)


,Sharpe,Omega,Riskiness
month vs week Spearman Corr.,-0.086154,-0.066923,-0.253077
